## Author

This notebook was authored by Amogh Mannekote (UFID 27146587).

## Index of questions (for referencing in the notebook)

1. Implement the elastic net TSP by minimizing the following objective function. \
 $E_{TSP}(P, Y) = \sum_{i = 1}^{N}{\sum_{a = 1}^M{P_{ia}||x_i - y_a||^2}} + \frac{\kappa}{2}\sum_{a = 1}^M{||y_a - y_{a \oplus 1}||_2^2}$
2. Document the settings of the free parameters $\kappa$, $\beta$ (the inverse temperature) and $M$, the number of hidden and actual cities.
3. If reducing $\kappa$, provide the rate, the total number of iterations, etc.
4. If steadily increasing $\beta$, provide the rate, the total number of iterations, etc.
5. Document initial conditions on $Y$.
6. Execute the elastic net on 100 cities in $[0,1]^2$ chosen using a uniform distribution.
7. Since we require average performance, please compute the average tour length for 100 trials and show a histogram of tour lengths.
8. Show visual examples of the shortest, longest and median length tours for the 100 city TSP.
9. [Extra Credit: 30 points] Compare against well known asymptotic lower bounds such as the ones uploaded on Canvas and the Held-Karp lower bound
10. [Extra Credit: 50 points] Execute the elastic net on the US 48 cities using data from TSPlib or from https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html. Give a detailed explanation
of the results
11. [Extra Credit: 50 points] Explain the equivalent objective function (free energy) shown below obtained by summing over all configurations of P
12. [Extra Credit: 70 points] Explain the relevance of the following objective function in the elastic net implementation
13. [Extra Credit: 50 points] Show the convergence of the algorithm by detailed plots of either (1) or (2) (depending on your implementation).

------------

## Imports

In [174]:
from itertools import chain, tee
from collections import deque
import math
import functools

import numpy as np

## Load the cities

In [175]:
from us_city import load_us_cities
cities, norm_cities = load_us_cities()

## Initialize hyperparameters

**NOTE TO TA: The following section is relevant for answering questions [2, 3, 4, 5].**

In [176]:
DIM = 2 # dimension of the points
N = cities.shape[0]  # number of cities
M = int(2.5 * N)  # number of intermediate points between the cities

KAPPA_START = 0.2  # starting value of kappa
GAMMA = 1.05  # kappa damping factor

BETA = 10 # inverse temperature

NUM_ITERS = 10000 # number of iterations of the algorithm
WORST_DISTANCE_EPSILON = 0.01

PERTURBATION_RADIUS = 1 # perturbation radius for generating the intermediate points

In [177]:
def init_intermediate_cities(self, strategy="centroid", **kwargs):
    if strategy == "centroid":
        self.intermediate_cities = np.tile(
            np.mean(self.cities, axis=0),
            M
        ).reshape((M, DIM))
        # Create perturbation vectors of magnitude radius
        perturbation_vectors = np.random.random((M, DIM))
        perturbation_vectors /= np.linalg.norm(perturbation_vectors, axis=1, ord=2, keepdims=True)
        perturbation_vectors *= PERTURBATION_RADIUS
        self.intermediate_cities += perturbation_vectors
    else:
        raise RuntimeError(f"Unknown strategy: {strategy}")

$$\kappa_{T+1} = \frac{\kappa_T}{\gamma}$$

In [178]:
def update_kappa(self):
    self.kappa = max(self.kappa / GAMMA, 0.01)

$$P_{ia} = \frac{exp\{-\beta\||x_i - y_a||^2_2\}}{\sum_{b=1}^{M}{exp\{-\beta||x_i - y_b||_2^2\}}}$$

In [179]:
def update_weights(self):
    self.diff = self.cities[:,np.newaxis] - self.intermediate_cities
    self.dist_squared = np.sum(self.diff ** 2, axis=-1)

    self.worst_dist = np.max(np.min(np.sqrt(self.dist_squared), axis=1))
    self.worst_dists.append(self.worst_dist)

    self.weights = np.exp(-self.dist_squared / (2 * (self.kappa ** 2)))
    self.weights /= self.weights.sum(axis=1)[:,np.newaxis]

$$d_a = \sum_{i = 1}^{N}{P_{ia}}$$

$$D_{aa} = d_a$$

In [180]:
def get_D(self):
    def d_ij(i, j):            
        if i == j:
            return np.sum(self.weights[:, int(j)])
        return 0
    return np.fromfunction(np.vectorize(d_ij), (M, M))

$$Y = (\kappa L + D)^{-1}P^TX$$

In [181]:
def update_intermediate_cities(self):
    self.intermediate_cities = np.linalg.inv((self.kappa * self.L + self.get_D())) @ self.weights.T @ self.cities

In [182]:
def decode_solution(self):
    neuron_city_pair = []

    for _ in range(self.dist_squared.shape[0]):
        # Pick the shortest edge among all edges between hidden and real cities.
        city, intermediate_city = np.unravel_index(np.argmin(self.dist_squared), self.dist_squared.shape)
        # Set distance to infinity so that it doesn't get picked again.
        self.dist_squared[city] = np.inf
        self.dist_squared[:, intermediate_city] = np.inf

        neuron_city_pair.append((intermediate_city, city))

    neuron_city_pair.sort(key=lambda x: x[0])
    print(f"neuron_city_pair: {neuron_city_pair}")
    self.neuron_city_pair = [x[1] for x in neuron_city_pair]


def compute_solution(self):
    assert len(self.neuron_city_pair) == N
    distance = 0
    tour = []
    for path_idx in range(len(self.neuron_city_pair)):
        tour.append((self.neuron_city_pair[path_idx], self.neuron_city_pair[(path_idx + 1) % N]))
        distance += np.linalg.norm(self.original_cities[path_idx] - self.original_cities[(path_idx + 1) % N])
    return distance, tour

In [183]:
def solve(self):
    for i in range(NUM_ITERS):
        self.update_weights()
        self.update_kappa()
        self.update_intermediate_cities()
        if i % 100 == 0:
            print(f"Iteration {i}, Worst distance: {self.worst_dist}")
        if self.worst_dist < WORST_DISTANCE_EPSILON:
            break
        if len(self.worst_dists) == 2:
            if self.worst_dists[1] - self.worst_dists[0] == 0:
                break
    self.decode_solution()
    return self.compute_solution()

In [184]:
class ElasticNet(object):
    
    def __init__(
        self,
        init_intermediate_cities,
        solve,
        update_weights,
        update_kappa,
        update_intermediate_cities,
        get_D,
        decode_solution,
        compute_solution
    ):
        # Take the functions as parameters to be able to split the
        # method definitions into separate cells.
        ElasticNet.init_intermediate_cities = init_intermediate_cities
        ElasticNet.solve = solve
        ElasticNet.update_weights = update_weights
        ElasticNet.update_kappa = update_kappa
        ElasticNet.update_intermediate_cities = update_intermediate_cities
        ElasticNet.get_D = get_D
        ElasticNet.decode_solution = decode_solution
        ElasticNet.compute_solution = compute_solution

        # Initialize kappa to the starting value
        self.kappa = KAPPA_START
        # Set original_cities to the cities
        self.original_cities = cities
        # Set cities to the normalized cities
        self.cities = norm_cities
        # Generate intermediate cities around the centroid
        self.init_intermediate_cities(strategy="centroid")
        # A queue to hold the last two worst distances (for the stopping criterion)
        self.worst_dists = deque(maxlen=2)


    @functools.cached_property
    def L(self):
        def L_ij(i, j, M):
            if i == j:
                return 2
            if i == (j + 1) % M:
                return 1
            if j == (i + 1) % M:
                return 1
            return 0
        return np.fromfunction(np.vectorize(L_ij), (M, M), M=M, dtype=int)

In [185]:
NUM_RUNS = 2
for run in range(NUM_RUNS):
    net = ElasticNet(
        init_intermediate_cities=init_intermediate_cities,
        solve=solve,
        update_weights=update_weights,
        update_kappa=update_kappa,
        update_intermediate_cities=update_intermediate_cities,
        get_D=get_D,
        decode_solution=decode_solution,
        compute_solution=compute_solution
    )
    distance, tour = net.solve()
    print(f"Distance {distance}, Tour: {tour}")

Iteration 0, Worst distance: 1.5079357376284546
neuron_city_pair: [(0, 9), (2, 2), (4, 7), (6, 12), (8, 15), (10, 18), (12, 20), (14, 17), (19, 19), (21, 13), (23, 11), (25, 3), (27, 4), (32, 21), (34, 0), (36, 1), (38, 6), (40, 5), (46, 10), (48, 8), (50, 16), (53, 14)]


NameError: name 'edges' is not defined

---------------

In [ ]:
net.worst_dist

0.11373388241244281

In [ ]:
net.weights[:5, :5]

array([[0.00000000e+00, 7.37102923e-19, 0.00000000e+00, 5.15461192e-01,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

In [ ]:
net.intermediate_cities[:3]

array([[-0.51263035, -0.22535008],
       [ 0.49996802,  0.31611335],
       [-0.48568822, -0.3583386 ]])

------------